#### 쿠팡 상품 검색 후, 상품별 가격 크롤링, 페이지 넘기기 가능
- https://selenium-python.readthedocs.io/installation.html(selenium 공식문서)

In [ ]:
# 불러올 라이브러리

import requests, time, csv, openpyxl
from datetime import datetime
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [ ]:
start = time.time()
#-----------------파일 저장하기 -------------------------------
wb = openpyxl.Workbook()
sheet1 = wb['Sheet']
sheet1.title = 'data'
sheet1.append(["name", "price"])
              
def crawling_product():
#----------------- 계정, 검색 상품, 검색할 페이지 지정-------------------------
    myid = input("ID를 적어주세요 >>")
    pwd = input("비밀번호를 적어주세요 >>")
    product = input("검색할 상품은 무엇인가요? >>")
    page_count = input("\n몇 페이지 정도 검색하실래요? >>")
    page_count = int(page_count)

#----------------- 쿠팡 로그인 ---------------------------
    chrome_driver = '/Users/1000/Downloads/chromedriver'
    driver = webdriver.Chrome(chrome_driver)

    driver.get('https://www.coupang.com/')
    search = driver.find_element_by_class_name('login').click()


    search_id = driver.find_element_by_id('login-email-input')
    search_id.send_keys(myid)

    search_pw = driver.find_element_by_id('login-password-input')
    search_pw.send_keys(pwd)
    search_pw.send_keys(Keys.RETURN)
        
    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.ID,'headerSearchKeyword')))

#----------------- 상품명과 page 별로 이동하면서 긁어오기-------------------------    
    page = 'https://www.coupang.com/np/search?q={}&brand=&offerCondition=&filter=&availableDeliveryFilter=&filterType=&isPriceRange=false&priceRange=&minPrice=&maxPrice=&page={}&trcid=&traid=&filterSetByUser=true&channel=user&backgroundColor=&component=&rating=0&sorter=scoreDesc&listSize=36'
    
    for i in range(1,page_count+1) :
        driver.get(page.format(product,i))
        
        src = driver.page_source
        soup = BeautifulSoup(src, 'html.parser')
        
        for b in soup.find_all('div', class_='descriptions-inner'):
            name = b.find('div', class_='name').get_text().strip()
            price = b.find('strong', class_='price-value').get_text().strip()
            
            sheet1.append([name, int(price.replace(',',''))])

        WebDriverWait(driver, 10)
        
    wb.save('{}_result.xlsx'.format(product))    
    
    return 
crawling_product()
print("\nWorkingTime: {} sec".format(time.time()-start))